# Implement ML pipeline Using the AWS Glue Workflow

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Create Resources](#Create-Resources)
1. [Build a Machine Learning Workflow](#Build-a-Machine-Learning-Workflow)
1. [Run the Workflow](#Run-the-Workflow)
1. [Clean Up](#Clean-Up)

<a id="Introduction"></a>
## Introduction

This notebook describes how to use Glue Workflow with PySpark scripts to create a machine learning pipeline across data preparation, model training, model evaluation and model register. The defintion of workflow as beflow:

<div align="center"><img width=300 src="images/glue_ml_workflow.png"></div>

## Setup

First, 

In [ ]:
import sys

# !{sys.executable} -m pip install 

### Import the Required Modules

In [15]:
import os
import sys
import uuid
import logging
import boto3
import sagemaker

from sagemaker.s3 import S3Uploader

sys.path.insert( 0, os.path.abspath("../common") )
import setup_iam_roles

session = sagemaker.Session()

region = boto3.Session().region_name
bucket = session.default_bucket()

id = uuid.uuid4().hex

# SageMaker Execution Role
sagemaker_execution_role = sagemaker.get_execution_role()

# Create a unique name for the AWS Glue job to be created. If you change the
# default name, you may need to change the Step Functions execution role.
glue_job_prefix = "customer-churn-etl"
glue_job_name = f"{glue_job_prefix}-{id}"

# Create a unique name for the AWS Lambda function to be created. If you change
# the default name, you may need to change the Step Functions execution role.
query_function_prefix = "query-evaluation-result"
query_function_name = f"{query_function_prefix}-{id}"

prefix = 'sagemaker/DEMO-xgboost-customer-churn'

Next, we'll create fine-grained IAM roles for the Lambda, Glue, and Step Functions resources. The IAM roles grant the services permissions within your AWS environment.

### Add permissions to your notebook role in IAM

The IAM role assumed by your notebook requires permission to create and run workflows in AWS Step Functions. If this notebook is running on a SageMaker notebook instance, do the following to provide IAM permissions to the notebook:

1. Open the Amazon [SageMaker console](https://console.aws.amazon.com/sagemaker/). 
2. Select **Notebook instances** and choose the name of your notebook instance.
3. Under **Permissions and encryption** select the role ARN to view the role on the IAM console.
4. Copy and save the IAM role ARN for later use. 
5. Choose **Attach policies** and search for `AWSGlueConsoleSageMakerNotebookFullAccess`.
6. Select the check box next to `AWSGlueConsoleSageMakerNotebookFullAccess` and choose **Attach policy**.

We also need to provide permissions that allow the notebook instance the ability to create an AWS Lambda function and AWS Glue job. We will edit the managed policy attached to our role directly to encorporate these specific permissions:

1. Under **Permisions policies** expand the AmazonSageMaker-ExecutionPolicy-******** policy and choose **Edit policy**.
2. Select **Add additional permissions**. Choose **IAM**  for Service and **PassRole** for Actions.
3. Under Resources, choose **Specific**. Select **Add ARN** and enter `query_training_status-role` for **Role name with path*** and choose **Add**. You will create this role later on in this notebook.
4. Select **Add additional permissions** a second time. Choose **Lambda** for Service, **Write** for Access level, and **All resources** for Resources.
5. Select **Add additional permissions** a final time. Choose **Glue** for Service, **Write** for Access level, and **All resources** for Resources.
6. Choose **Review policy** and then **Save changes**.

Create an IAM role for Glue Job
* Providing access on the S3 bucket
* Executing SageMaker training job and model deployment

In [3]:
glue_role_name = "AWS-Glue-S3-SageMaker-Access"
glue_role_arn = setup_iam_roles.create_glue_role(glue_role_name, bucket)
glue_role_arn

Using ARN from existing role: AWS-Glue-S3-SageMaker-Access


'arn:aws:iam::452533547478:role/AWS-Glue-S3-SageMaker-Access'

### Prepare the Dataset
This notebook uses the XGBoost algorithm to automate the classification of unhappy customers for telecommunication service providers. The goal is to identify customers who may cancel their service soon so that you can entice them to stay. This is known as customer churn prediction.

The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.

In [4]:
train_prefix = "train"
val_prefix = "validation"
test_prefix = "test"

raw_data = f"s3://{bucket}/{prefix}/input"
processed_data = f"s3://{bucket}/{prefix}/processed"

train_data = f"{processed_data}/{train_prefix}/"
validation_data = f"{processed_data}/{val_prefix}/"
test_data = f"{processed_data}/{test_prefix}/"

Upload data to `S3 Bucket`

In [5]:
S3Uploader.upload(
    local_path="../data/churn_processed.csv",
    desired_s3_uri=f"{raw_data}",
    sagemaker_session=session,
)

's3://sagemaker-ap-southeast-2-452533547478/sagemaker/DEMO-xgboost-customer-churn/input/churn_processed.csv'

## Build a Machine Learning Workflow

We are going to use Glue Workflow as the orchestration engine, Glue Job for the data preprocessing and model training/deployment as the steps

* [**Glue Workflow**](https://docs.aws.amazon.com/glue/latest/dg/workflows_overview.html) - Orchestration engine for ML workflow.
* [**Glue Job**](https://docs.aws.amazon.com/glue/latest/dg/author-job.html) - Business logic for ETL or python shell.
* [**Glue Trigger**](https://docs.aws.amazon.com/glue/latest/dg/trigger-job.html) - Triggers Glue Job as steps.

### Create AWS Glue Workflow

#### Create Glue Workflow Object


In [6]:
glue_client = boto3.client("glue")

In [21]:
glue_workflow_name = f"CustomerChurnMLWorkflow-{id}"
response = glue_client.create_workflow(
    Name=glue_workflow_name,
    Description='AWS Glue workflow to process data and create training jobs'
)

#### Create Glue Jobs 

In [38]:
# Data Processing Job
data_processing_script_path = S3Uploader.upload(
    local_path="../common/glue_preprocessing.py",
    desired_s3_uri=f"s3://{bucket}/{prefix}/glue/scripts",
    sagemaker_session=session,
)
data_processing_job_name = "DataProcessingJob"
response = glue_client.create_job(
    Name=data_processing_job_name,
    Description='Preparing data for SageMaker training',
    Role=glue_role_arn,
    ExecutionProperty={
        'MaxConcurrentRuns': 2
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': data_processing_script_path,
    },
    DefaultArguments={
        "--job-bookmark-option": "job-bookmark-enable",
        "--enable-metrics": "",
        "--additional-python-modules": "pyarrow==2,awswrangler==2.9.0,fsspec==0.7.4"
    },
    MaxRetries=0,
    Timeout=60,
    MaxCapacity=10.0,
    GlueVersion='2.0'
)

In [45]:
# Model Training & Deployment Job
model_training_deployment_script_path = S3Uploader.upload(
    local_path="./code/model_training_deployment.py",
    desired_s3_uri=f"s3://{bucket}/{prefix}/glue/scripts",
    sagemaker_session=session
)

model_training_deployment_job_name = "ModelTrainingDeploymentJob"
response = glue_client.create_job(
    Name=model_training_deployment_job_name,
    Description='Model training and deployment',
    Role=glue_role_arn,
    ExecutionProperty={
        'MaxConcurrentRuns': 2
    },
    Command={
        'Name': 'pythonshell',
        'ScriptLocation': model_training_deployment_script_path,
        'PythonVersion': '3'
    },
    DefaultArguments={
        "--job-bookmark-option": "job-bookmark-enable",
        "--enable-metrics": ""
    },
    MaxRetries=0,
    Timeout=60,
    MaxCapacity=1,
    GlueVersion='1.0'
)

In [50]:
# quick test
response = glue_client.start_job_run(
    JobName=model_training_deployment_job_name,
    Arguments={
        '--train_input_path': processed_data,
        '--model_output_path': model_output_path,
        '--algorithm_image': image_uri,
        '--role_arn': sagemaker_execution_role,
    },
    MaxCapacity=1
)


#### Create Glue Triggers

In [33]:
response = glue_client.create_trigger(
    Name='TriggerDataProcessingJob',
    Description='Triggering Data Processing Job',
    Type='ON_DEMAND',
    WorkflowName=glue_workflow_name,
    Actions=[
        {
            'JobName': data_processing_job_name,
            'Arguments': {
                '--INPUT_DIR': raw_data,
                '--PROCESSED_DIR': processed_data
            },
        },
    ]
)


In [34]:
model_output_path = f"s3://{bucket}/{prefix}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
)

processed_data, sagemaker_execution_role, image_uri, model_output_path

('s3://sagemaker-ap-southeast-2-452533547478/sagemaker/DEMO-xgboost-customer-churn/processed',
 'arn:aws:iam::452533547478:role/service-role/AmazonSageMaker-ExecutionRole-20210709T125430',
 '783357654285.dkr.ecr.ap-southeast-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
 's3://sagemaker-ap-southeast-2-452533547478/sagemaker/DEMO-xgboost-customer-churn/output')

In [43]:
response = glue_client.create_trigger(
    Name='TriggerModelTrainingDeploymentJob',
    Description='Triggering Model Training Deployment Job',
    WorkflowName=glue_workflow_name,
    Type='CONDITIONAL',
    StartOnCreation=True,
    Predicate={
        'Conditions': [
            {
                'LogicalOperator': 'EQUALS',
                'JobName': data_processing_job_name,
                'State': 'SUCCEEDED'
            },
        ]
    },
    Actions=[
        {
            'JobName': model_training_deployment_job_name,
            'Arguments': {
                '--train_input_path': processed_data,
                '--model_output_path': model_output_path,
                '--algorithm_image': image_uri,
                '--role_arn': sagemaker_execution_role,
            }
        }
    ]
)


#### Start Workflow Run

## Run the Workflow
Create your workflow using the workflow definition above, and render the graph with [render_graph](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.render_graph):

In [51]:
response = glue_client.start_workflow_run(
    Name=glue_workflow_name,
#     RunProperties={
#         'string': 'string'
#     }
)

## Clean Up
When you are done, make sure to clean up your AWS account by deleting resources you won't be reusing. Uncomment the code below and run the cell to delete the Glue job, Lambda function, and Step Function.

In [ ]:
# deletion
response = glue_client.delete_workflow(
    Name=glue_workflow_name
)
